## Imports of dependencies and print GPU specs

In [ ]:
from transformers import AutoTokenizer
import torch
import deepspeed
import pandas as pd
from datasets import Dataset, load_dataset, load_from_disk
from transformers import AutoTokenizer

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))
print(deepspeed.__version__)


[2025-06-09 11:21:16,445] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0609 11:21:17.721000 1044 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


True
1
NVIDIA GeForce RTX 5090
0.16.5


## Loading dataset and split it into trin and test.

In [2]:
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", 'en')
dataset = dataset['train'].train_test_split(test_size=0.1)
print(len(dataset["train"]))
print(len(dataset["test"]))
print(dataset["test"].to_pandas().head())


17733
1971
                                            Question  \
0  A 24-year-old woman presents with a 1-day hist...   
1  A 35-week pregnant woman has an ultrasound wit...   
2  What is the most predictive and dangerous side...   
3  A 4-year-old boy presents with a peeling eryth...   
4  A 74-year-old right-handed woman presents with...   

                                         Complex_CoT  \
0  Alright, so we've got a young woman here, just...   
1  Okay, so we've got a woman who's 35 weeks preg...   
2  Alright, let's think about this for a second. ...   
3  A 4-year-old boy comes in with a rash that's p...   
4  Okay, let's think this through. We have a 74-y...   

                                            Response  
0  Based on the clinical findings you've shared, ...  
1  In this clinical situation, the appropriate ma...  
2  The most predictive and dangerous side effect ...  
3  The condition described fits the clinical pict...  
4  The clinical presentation and imaging

## Select Hugging Face model and its tokenizer

In [ ]:
model_name = "nvidia/Nemotron-Research-Reasoning-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

## Function to tokenize and format the data

In [ ]:
def preprocess_function(examples):

    prompts = [f"[INST] {question} [/INST]" for question in examples['Question']]
    responses = examples['Response']
    
    texts = []
    for prompt, response in zip(prompts, responses):
        texts.append(f"{prompt} {response}")
    
    print(texts[:1])

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )
    
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

## All dataset is tokenized

In [5]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

Map:   0%|          | 0/17733 [00:00<?, ? examples/s]

["[INST] What is the most probable diagnosis for a 30-year-old female with a painful, red, warm nail fold for the past 7 days, who has been using home remedies such as turmeric powder and warm saline compressions without relief? [/INST] The most probable diagnosis for the 30-year-old female with a painful, red, and warm nail fold persisting for 7 days, despite using home remedies like turmeric powder and warm saline compressions, is paronychia. Paronychia is a common infection of the nail fold, often caused by bacteria entering through small cuts or breaks in the skin. Given the symptoms and lack of relief from anti-inflammatory home remedies, it's likely that this condition may require medical treatment, potentially including antibiotics, to effectively address the infection."]
['[INST] In milk, which substance(s) does not directly or indirectly possess anti-infective properties? [/INST] In milk, the substance(s) that does not directly or indirectly possess anti-infective properties i

Map:   0%|          | 0/1971 [00:00<?, ? examples/s]

["[INST] A 24-year-old woman presents with a 1-day history of nausea and weakness, has a temperature of 38.4°C (101°F), and exhibits right costovertebral angle tenderness. She is sexually active with two male partners and uses an oral contraceptive, with inconsistent condom use. Her last menstrual period was four days ago, and pelvic examination is normal. Based on these clinical findings, what is the most likely cause of her symptoms? [/INST] Based on the clinical findings you've shared, the most likely cause of this young woman's symptoms is acute pyelonephritis. The combination of her fever, nausea, weakness, and right costovertebral angle tenderness strongly points towards an infection of the kidneys. Her history of being sexually active with inconsistent condom use increases the risk of urinary tract infections, which can ascend to the kidneys, causing pyelonephritis. The normal pelvic exam helps rule out pelvic inflammatory disease, focusing the diagnosis on the renal system. Giv

In [7]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer
import torch

## Create model shell from specified Hugging Face model's name, and using available GPU.

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Using WanDB to monitor learning process

In [9]:
def try_import_wandb():
    try:
        import wandb
        return True
    except ImportError:
        return False

## Use the best hyperparameters to create trining argument set

In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen-1.5B-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=1000,
    save_total_limit=1,
    eval_steps=500,
    logging_dir="./logs",
    logging_steps=50,
    learning_rate=2.6563467269626478e-05,
    weight_decay=0.01,
    warmup_steps=200,
    lr_scheduler_type="cosine",
    report_to="wandb" if try_import_wandb() else "none",
    fp16=False,
    bf16=True,
    gradient_checkpointing=True
)

# Create trainer objet defining the model, arguments train and test data.

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

In [12]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: luis-orellana777 (luis-orellana777-sngular) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
50,5.759800
100,0.938000
150,0.791500
200,0.766000
250,0.741700
300,0.739900
350,0.716400
400,0.722000
450,0.716600
500,0.699500


TrainOutput(global_step=2770, training_loss=0.7691792026753891, metrics={'train_runtime': 5374.4353, 'train_samples_per_second': 16.498, 'train_steps_per_second': 0.515, 'total_flos': 4.198444485480284e+17, 'train_loss': 0.7691792026753891, 'epoch': 4.99233198015336})

## Save the fine tune model and its tokenizer locally.

In [23]:
trainer.save_model("./model-finetuned-medical-reasoning-qwen-1.5B")

In [22]:
tokenizer.save_pretrained("./model-finetuned-medical-reasoning-qwen-1.5B")

('./model-finetuned-medical-reasoning-qwen-1.5B\\tokenizer_config.json',
 './model-finetuned-medical-reasoning-qwen-1.5B\\special_tokens_map.json',
 './model-finetuned-medical-reasoning-qwen-1.5B\\tokenizer.json')

## Save the fine tune model and its tokenizer into Hugging Face.

In [15]:
trainer.push_to_hub("Luis-Orellana777/model-finetuned-medical-reasoning-qwen-1.5B")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.65k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Luis-Orellana777/qwen-1.5B-finetuned/commit/69a14f132df8588623159345f958e60f4e284b79', commit_message='Luis-Orellana777/model-finetuned-medical-reasoning-qwen-1.5B', commit_description='', oid='69a14f132df8588623159345f958e60f4e284b79', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Luis-Orellana777/qwen-1.5B-finetuned', endpoint='https://huggingface.co', repo_type='model', repo_id='Luis-Orellana777/qwen-1.5B-finetuned'), pr_revision=None, pr_num=None)

In [16]:
tokenizer.push_to_hub("Luis-Orellana777/model-finetuned-medical-reasoning-qwen-1.5B")

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Luis-Orellana777/model-finetuned-medical-reasoning-qwen-1.5B/commit/0d856a98dd5067f3a464b34fb9602cf0e7a4afe1', commit_message='Upload tokenizer', commit_description='', oid='0d856a98dd5067f3a464b34fb9602cf0e7a4afe1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Luis-Orellana777/model-finetuned-medical-reasoning-qwen-1.5B', endpoint='https://huggingface.co', repo_type='model', repo_id='Luis-Orellana777/model-finetuned-medical-reasoning-qwen-1.5B'), pr_revision=None, pr_num=None)

## Tokenize the prompt, and select generation parameters to make predictions

In [ ]:
def generate_response(prompt, model, tokenizer, max_length=512):
    input_ids = tokenizer.encode(f"[INST] {prompt} [/INST]", return_tensors="pt").to(model.device)
    attention_mask = (input_ids != tokenizer.pad_token_id).long().to(model.device)
    
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=max_length,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.pad_token_id
    )
    
    return tokenizer.decode(output[0], skip_special_tokens=True)

## Sends medical symptoms and print the diagnosis.

In [18]:
test_prompt = "Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?"
response = generate_response(test_prompt, model, tokenizer)
print(response)

c:\Users\Luis\anaconda3\envs\LLM\lib\site-packages\torch\utils\checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings? [/INST] The symptoms you've described—sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg—are indicative of a condition known as "left limb hemiplegia." This condition is often associated with a cardiac issue involving the left ventricle. 

In the context of cardiac abnormalities, the most specific abnormality that could explain these symptoms is an "abnormal left ventricular hypertrophy." This condition occurs when the left ventricle becomes enlarged, which can lead to a variety of symptoms, including muscle weakness in the affected limb and the presence of swelling and tenderness in the affected leg. 

Therefore, the most specific cardiac abn

In [ ]:
test_prompt = "Given the symptoms, explain these findings: Mild memory loss. Motor incoordination. Patient is 69 years old. Sleep apnea. Attention deficit. Temperament and mood swings."
response = generate_response(test_prompt, model, tokenizer)
print(response)

[INST] Given the symptoms, explain these findings:Mild memory loss. Motor incoordination. Patient is 69 years old. Sleep apnea. Attention deficit. Temperament and mood swings. [/INST] The symptoms you've described are highly suggestive of a condition known as narcolepsy, which is characterized by a combination of symptoms such as mild memory loss, motor incoordination, attention deficit, mood swings, and sleep disturbances. These symptoms are particularly notable in older adults, which aligns with the age of 69 years.

Narcolepsy is an autoimmune disorder that primarily affects the central nervous system. The patient's sleep apnea, combined with their age, is a key indicator of this condition. The mild memory loss and motor incoordination suggest possible neurological issues, while the attention deficit and mood swings point to potential neurological deficits.

In terms of diagnosis, narcolepsy can be evaluated through a combination of clinical examination, blood tests, and imaging stu

In [24]:
test_prompt = "A patient presents with a fever and a persistent cough for the last three days. What are the possible differential diagnoses?"
response = generate_response(test_prompt, model, tokenizer)
print(response)

c:\Users\Luis\anaconda3\envs\LLM\lib\site-packages\torch\utils\checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[INST] A patient presents with a fever and a persistent cough for the last three days. What are the possible differential diagnoses? [/INST] The symptoms of a persistent cough with a fever for three days can suggest several potential diagnoses, each requiring careful evaluation and appropriate management:

1. **Chronic Influenza**: This is a common cause of persistent cough and fever, particularly in the winter months. It can lead to a prolonged course, which fits the patient's symptoms.

2. **Pneumocystis Jiroveci (Pneumocystis deltemata)**: This is a bacterial infection that can cause severe cough and fever, often leading to a prolonged course. It is a less common cause compared to influenza, especially in the winter.

3. **Cystic Fibrosis**: This is a genetic condition where the immune system is compromised, leading to chronic cough and fever. It is less common and typically occurs in older adults.

4. **Hypersensitive Cough**: This can occur due to various underlying causes, includ

## Prints default generation parameters which are used by the fine tuned model.

In [1]:
from transformers import AutoModelForCausalLM

LOCAL_MODEL_PATH = "./model-finetuned-medical-reasoning-qwen-1.5B"

print(f"Loading model from: {LOCAL_MODEL_PATH}")

try:
    model = AutoModelForCausalLM.from_pretrained(LOCAL_MODEL_PATH)

    config = model.generation_config

    
    temperature = config.temperature
    top_p = config.top_p
    top_k = config.top_k
    repetition_penalty = config.repetition_penalty
    max_new_tokens = config.max_new_tokens

    print(f"Temperature: {temperature}")
    print(f"Top P: {top_p}")
    print(f"Top K: {top_k}")
    print(f"Repetition Penalty: {repetition_penalty}")
    print(f"Max New Tokens: {max_new_tokens}")

except OSError as e:
    print(f"\nError: Could not find model at the specified path.")
    print(f"Please make sure '{LOCAL_MODEL_PATH}' is the correct directory.")
    print(f"Details: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Loading model from: ./model-finetuned-medical-reasoning-qwen-1.5B
[2025-06-18 23:37:04,933] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


W0618 23:37:05.540000 27768 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Temperature: 1.0
Top P: 1.0
Top K: 50
Repetition Penalty: 1.0
Max New Tokens: None
